# 读取数据

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('test.csv',encoding='gbk')
df.info()

# 顺序基本一致

In [ ]:
df_drop = df.dropna(axis=0)
df_drop.info()

In [ ]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(df_drop['Y']))}
class_mapping

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_drop['Y'] = df_drop['Y'].map(class_mapping)
y = LabelEncoder().fit_transform(df_drop['Y'].values)
X = df_drop.iloc[:, 0:13].values #自己根据情况来
print(y.shape)
print(X.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_std = stdsc.fit_transform(X)
X1_std = stdsc.fit_transform(X1)
print(X_std.shape)

# 调用保存的训练模型

In [ ]:
from sklearn.externals import joblib
model_RF = joblib.load('train.pkl')

# 用保存的模型预测并作图

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from scipy import interp
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
cv = StratifiedKFold(n_splits=5)
classifier = model_RF
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
i = 0
lw = 2
probas_ = model_RF.predict_proba(X_std)
# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(y, probas_[:, 1])
fpr=fpr
tpr=tpr
roc_auc = auc(fpr, tpr)
aucs.append(roc_auc)
#plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
plt.plot(fpr, tpr, color='b', alpha=.8,lw=lw, label='ROC (area = %0.2f)' % roc_auc) 
plt.plot([0, 1], [0, 1], color='r', lw=lw, linestyle='--')
plt.xlim([-0, 1])
plt.ylim([-0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Independent Validation')
plt.legend(loc="lower right" ,
           #fontsize = 'xx-small'
          )
#plt.savefig('RMMM_ROC4.jpg',dpi=1200) #这个是保存的意思，句首的#去掉就可以运行。
plt.show()

# 预测每个样本的值的概率

In [ ]:
probas_ = model_RF.predict_proba(X_std)   #根据需求换X_std

# 将每次结果输出

In [ ]:
probas_

# 将结果保存成Excel表格

In [ ]:
data = pd.DataFrame(probas_)
writer = pd.ExcelWriter('1.xlsx')  #表格命名，保存在当下文件夹里
data.to_excel(writer, 'page_1', float_format='%.5f')
writer.save()